<a href="https://colab.research.google.com/github/kkarimwahba/colab/blob/main/knowledge_graph_nlp_tutorial_bert_spacy_nltk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import re
import pandas as pd
import bs4
import requests
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')

from spacy.matcher import Matcher
from spacy.tokens import Span

import networkx as nx

import matplotlib.pyplot as plt
from tqdm import tqdm

pd.set_option('display.max_colwidth', 200)
%matplotlib inline

In [3]:
# import wikipedia sentences
candidate_sentences = pd.read_csv("/content/wiki_sentences_v2.csv")
candidate_sentences.shape

(4318, 1)

In [4]:
!pip install pytorch-pretrained-bert pytorch-nlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 11.5 MB/s eta 0:00:00


In [5]:
pip install tensorflow opencv-python-headless

In [6]:
!pip install tensorflow


In [7]:
!pip install keras


In [8]:
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [9]:
# Import Libraries

import tensorflow as tf
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
df = pd.read_csv("/content/drive/MyDrive/cola_public/raw/in_domain_train.tsv",
                 delimiter='\t', header=None,
                 names=['sentence_source', 'label', 'label_notes', 'sentence'])

In [12]:
df.shape

(8551, 4)

In [13]:
df.sample(10)

,sentence_source,label,label_notes,sentence
1595,r-67,1,NaN,webster touched a sword after henry had done so .
7525,sks13,1,NaN,the girls likes themselves .
6383,d_98,1,NaN,you may pick any flower except the rose .
5347,b_73,1,NaN,they think she has too much independence .
5968,c_13,1,NaN,i have eaten the beef waffles .
6834,m_02,1,NaN,tatiana wrote to onegin .
852,bc01,0,*,"mary came to read fred 's story , and i also came to ."
129,cj99,1,NaN,"how much harder has it rained , when you see a faster flow in the river ?"
6449,d_98,1,NaN,confiscate any liquor .
3227,l-93,0,*,the romans destroyed the city into ruins .


In [14]:
# Create sentence and label lists
sentences = df.sentence.values

# We need to add special tokens at the beginning and end of each sentence for BERT to work properly
sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
labels = df.label.values

In [15]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print ("Tokenize the first sentence:")
print (tokenized_texts[0])

100%|██████████| 231508/231508 [00:00<00:00, 2009771.21B/s]


Tokenize the first sentence:
['[CLS]', 'our', 'friends', 'wo', 'n', "'", 't', 'buy', 'this', 'analysis', ',', 'let', 'alone', 'the', 'next', 'one', 'we', 'propose', '.', '[SEP]']


In [16]:
# Set the maximum sequence length. The longest sequence in our training set is 47, but we'll leave room on the end anyway.
# In the original paper, the authors used a length of 512.
MAX_LEN = 128

In [17]:
# Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

In [18]:
# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [19]:
# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

In [20]:
# Use train_test_split to split our data into train and validation sets for training

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels,
                                                            random_state=2018, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.1)

In [21]:
# Convert all of our data into torch tensors, the required datatype for our model

train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [22]:
# Select a batch size for training. For fine-tuning BERT on a specific task, the authors recommend a batch size of 16 or 32
batch_size = 32

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop,
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [23]:
# Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top.

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

100%|██████████| 407873900/407873900 [00:07<00:00, 53707341.02B/s]


In [24]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [{'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],'weight_decay_rate': 0.01},
                                {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],'weight_decay_rate': 0.0}]

In [25]:
# This variable contains all of the hyperparemeter information our training loop needs
optimizer = BertAdam(optimizer_grouped_parameters,lr=2e-5,warmup=.1)

In [26]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [27]:
!pip install wandb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.0/254.0 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.9 MB/s eta 0:00:00


In [28]:
def ret_dataloader():
    # Implement your data loading logic here
    # This is just a placeholder; replace it with your actual data loading code
    train_dataset = YourDatasetClassForTraining()
    val_dataset = YourDatasetClassForValidation()

    train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

    return train_dataloader, val_dataloader


In [29]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import wandb

# Define your model architecture
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        # Example: Linear layer with input size 10 and output size 2
        self.fc = nn.Linear(10, 2)

    def forward(self, x):
        return self.fc(x)

class YourDatasetClass(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file)
        # Add any additional preprocessing or setup steps here

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Implement how to retrieve a sample based on the index (idx)
        # Return a dictionary or tuple containing features and labels
        sample = {
            'feature': ...,
            'label': ...
        }
        return sample

def ret_model():
    return MyModel()

def ret_dataloader():
    dataset = YourDatasetClass('/content/wiki_sentences_v2.csv')
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size

    train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

    train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

    return train_dataloader, val_dataloader

def train(model, train_dataloader, optimizer, criterion, device):
    # Implement your training loop
    pass

def validate(model, val_dataloader, criterion, device):
    # Implement your validation loop
    pass

# Initialize WandB
wandb.init()

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# Retrieve model and move it to the device
model = ret_model()
model.to(device)

# Check if the model has parameters
if list(model.parameters()):
    # If there are parameters, create the optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()
else:
    # If there are no parameters, print a message or handle accordingly
    print("No trainable parameters found in the model.")

# Retrieve dataloaders
train_dataloader, validation_dataloader = ret_dataloader()

# Training loop
num_epochs = 10  # Adjust as needed
for epoch in range(num_epochs):
    # Train the model
    train_loss = train(model, train_dataloader, optimizer, criterion, device)

    # Validate the model
    val_loss = validate(model, validation_dataloader, criterion, device)

    # Log metrics to WandB
    wandb.log({"train_loss": train_loss, "val_loss": val_loss})

# Save the model if needed
torch.save(model.state_dict(), "your_model.pth")


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: ERROR API key must be 40 characters long, yours was 51


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


cpu


In [32]:
df = pd.read_csv("/content/drive/MyDrive/cola_public/raw/in_domain_dev.tsv"
                , delimiter='\t', header=None,
                 names=['sentence_source', 'label', 'label_notes', 'sentence'])

In [33]:
# Create sentence and label lists
sentences = df.sentence.values

# We need to add special tokens at the beginning and end of each sentence for BERT to work properly
sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
labels = df.label.values

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]


MAX_LEN = 128

# Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(labels)

batch_size = 32


prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [34]:
import torch
import torch.nn as nn
from transformers import BertModel

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.bert_model = BertModel.from_pretrained('bert-base-uncased')  # Replace with your desired BERT model

    def forward(self, input_ids, token_type_ids, attention_mask):
        outputs = self.bert_model(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)
        # Your additional layers and computations go here
        return outputs

# Example input data (replace with your actual data)
input_ids = torch.tensor([[1, 2, 3, 4, 5], [6, 7, 8, 9, 10]])
token_type_ids = torch.tensor([[0, 0, 1, 1, 1], [0, 0, 0, 1, 1]])
attention_mask = torch.tensor([[1, 1, 1, 1, 1], [1, 1, 1, 1, 1]])

model = MyModel()

# Perform the forward pass
with torch.no_grad():
    logits = model(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [35]:
from transformers import get_linear_schedule_with_warmup

def ret_scheduler(optimizer, dataloader, num_epochs, warmup_proportion=0.1):
    total_steps = len(dataloader) * num_epochs
    warmup_steps = int(total_steps * warmup_proportion)

    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=warmup_steps,
        num_training_steps=total_steps
    )

    return scheduler


In [36]:
!pip install gtts SpeechRecognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 14.4 MB/s eta 0:00:00


In [37]:
!sudo apt-get install portaudio19-dev


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libportaudio2 libportaudiocpp0
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 23 not upgraded.
Need to get 188 kB of archives.
After this operation, 927 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudiocpp0 amd64 19.6.0-1.1 [16.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 portaudio19-dev amd64 19.6.0-1.1 [106 kB]
Fetched 188 kB in 0s (458 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEn

In [38]:
!pip install pyaudio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 748.6 kB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyaudio: filename=PyAudio-0.2.14-cp310-cp310-linux_x86_64.whl size=63860 sha256=592c36c01288747df44e88c59c64beec6bc1ab172c81b45e5ba8b7476b8859de
  Stored in directory: /root/.cache/pip/wheels/d6/21/f4/0b51d41ba79e51b16295cbb096ec49f334792814d545b508c5
Successfully built pyaudio


In [43]:
import speech_recognition as sr
from gtts import gTTS
import os

# Speech recognition setup
recognizer = sr.Recognizer()

# Function to recognize speech
def recognize_speech():
    with sr.Microphone() as source:
        print("Say something:")
        recognizer.adjust_for_ambient_noise(source)
        audio_data = recognizer.listen(source)

    try:
        user_input = recognizer.recognize_google(audio_data)
        print("You:", user_input)
        return user_input
    except sr.UnknownValueError:
        print("Google Speech Recognition could not understand audio")
        return None
    except sr.RequestError as e:
        print(f"Could not request results from Google Speech Recognition service; {e}")
        return None

# Function to generate speech
def generate_speech(text):
    tts = gTTS(text=text, lang='en')
    tts.save('output_voice.mp3')
    os.system('mpg123 output_voice.mp3')  # You may need to install mpg123 for playing audio

# Example conversation loop
while True:
    # Recognize user's speech
    user_input = recognize_speech()

    # Check if speech was recognized
    if user_input:
        # Process user's input (add your logic here)
        # For example, you can use a chatbot model to generate a response

        # Generate a response
        response = "I heard you say: " + user_input

        # Output the response
        print("Bot:", response)

        # Convert the response to speech and play it
        generate_speech(response)


OSError: ignored